<h1>Analisis Sentimen Terhadap Presiden Jokowi</h1>

In [92]:
import re as regex
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

<p>Data berikut diperoleh dari komentar warganet terhadap Presiden Jokowi di Instagram. Pengumpulan data dilakukan dengan melakukan pencarian berdasarkan tag #jokowi</p>

In [94]:
data = pd.read_csv("dataset jokowi.csv", encoding = "ISO-8859-1")

arr_data = []
for index, row in data.iterrows():
    map_data = {}
    r = row[0].split(";")
    map_data["label"] = r[0]
    map_data["kalimat"] = r[1]
    
    if not pd.isnull(row[1]):
        map_data["kalimat"] += row[1]
    arr_data.append(map_data)
print(arr_data[19])

{'label': 'pos', 'kalimat': 'MAJU TRUSS PAK JOKOWI!!!'}


<h2><i>Preprocessing</i></h2>

<h4>Symbol Removal</h4>

In [95]:
symbolless = []
for d in arr_data:
    sentence = regex.sub("[^a-zA-Z0-9 ]","",d["kalimat"])
    symbolless.append(sentence)
print(symbolless[19])

MAJU TRUSS PAK JOKOWI


<h4>Casefolding</h4>

In [96]:
cases = []
for s in symbolless:
    sentence = s.lower()
    cases.append(sentence)
print(cases[19])

maju truss pak jokowi


<h4>Stopword Removal</h4>

In [111]:
factory = StopWordRemoverFactory()
stopwords = factory.create_stop_word_remover()

removed_cases = []

for c in cases:
    c = stopwords.remove(c)
    print(c)

segala difoto biar apa
pencitraan terus
datang cuman pelanga plongo
lambatnya bertindak
datang cuma foto doang
semangat pak semua mendukung bapak
sehat terus pak tetap berjiwa besar selalu jadi sosok penyabar atas nyinyiran orang yg tak tau apa apa dirimu
kalo emang indonesia mau maju pikiran warganya ga sempit masa presiden sendiri ngebakar wilayahnya sendiri
wkwkwkwk mantap jok kali aktingnya gak sendirian nih yeee
kalau foto foto doang semua orang jugha kayak gitu bilang prestasi haha lucunya negriku
mundur pakgak kan bapak urus negara iniserahkan ke ahli nya
cekrek nunduk dikit cekrek hm wkwk pencitraan yg sangat unfaedah
ngapain pa situ kerja nya cuman pencitraan malu pemimpin paling buruk sejarah indonesia
presiden tukang bohong dipercaya heran guamah
datang cuma mau foto mending ga usah datang paak
cuma era bapak doang negara kacau
masyaallah pak fighting
tuhan punya rencana lebih indah pada semua
stop menyindir nyarik kesalahan org intropeksi diri tidak tahu mna berusaha gbu
ma

<h4>Tokenization</h4>

In [98]:
tokens = []

for c in cases:
    token_data = c.split(" ")
    tokens.append(token_data)
print(tokens[19])

['maju', 'truss', 'pak', 'jokowi']


<h4>Stemming</h4>

In [99]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

stem_word = []

for t in tokens:
    comment = []
    for w in t:
        stem_data = stemmer.stem(w)
        comment.append(stem_data)
    stem_word.append(comment)
print(stem_word[19])

['maju', 'truss', 'pak', 'jokowi']


#### Slang Word

In [100]:
kalimat = []
nama= 'jokowi'  
for s in symbolless:
    sentence = s.lower()
    kalimat.append(sentence)

sentence_list = kalimat[19].split()


for word in sentence_list:
        if kata_dituju in word:
            b = 1
        else:
            b = 0
if b == 1:
    print('Kata Jokowi Disebut')
else:
    print('Tidak ada nama Jokowi')

Kata Jokowi Disebut


In [101]:
slang = []
Kata= {"truss": "terus"}   
for s in symbolless:
    sentence = s.lower()
    slang.append(sentence)

sentence_list = slang[19].split()
new_sentence = []

for word in sentence_list:
    for candidate_replacement in Kata:
        if candidate_replacement in word:
            word = word.replace(candidate_replacement, Kata[candidate_replacement])

    new_sentence.append(word)

rfrm = " ".join(new_sentence)
print(rfrm)

maju terus pak jokowi


<h2>Naive Bayes</h2>

<p>Corpus untuk proses Naive Bayes</p>

In [102]:
nb_corpus = []

In [103]:

ALPHA = 1

def search_corpus(word):
    for i in range(len(nb_corpus)):
         if nb_corpus[i]["kata"] == word:
            return nb_corpus[i], i
    return({}, -1)

def naive_bayes_formula(posneg_number,total_in_corpus):
    if total_in_corpus == 0:
        return 0
    return((posneg_number + ALPHA)/total_in_corpus)

def add_into_corpus(data):
    search_result, index = search_corpus(data['kata'])
    if not search_result:
        nb_corpus.append(data)
    else:
        nb_corpus[index]["pos"] += 1
        nb_corpus[index]["neg"] += 1
        if data["pos"] > data["neg"]:
            nb_corpus[index]["neg"] -= 1
        elif data["neg"] > data["pos"]:
            nb_corpus[index]["pos"] -= 1

def naivebayes(single_data):
    arr_data = []
    
    total_pos = 1
    total_neg = 1
    
    for word in single_data:
        data_in_corpus, index = search_corpus(word)
        pos = 1
        neg = 1
        if data_in_corpus:
            pos = naive_bayes_formula(data_in_corpus["pos"], (data_in_corpus["pos"] + data_in_corpus["neg"]))
            neg = naive_bayes_formula(data_in_corpus["neg"], (data_in_corpus["pos"] + data_in_corpus["neg"]))
        add_into_corpus({
        "kata": word, 
        "pos": pos,
        "neg": neg
        })
        total_pos *= pos
        total_neg *= neg
    
    label = "positive"
    
    if total_neg > total_pos:
        label = "negative"
    elif total_neg == total_pos:
        label = "neutral"
        
    print(single_data,"     ",label)

<p>Training dilakukan dengan menggunakan 10 data positif dan negatif</p>

In [104]:
training_data = stem_word[0:7] + stem_word[16:17]
training_raw_data = arr_data[0:7] + arr_data[16:17]
print(training_raw_data)

[{'label': 'neg', 'kalimat': 'segala difoto biar apa'}, {'label': 'neg', 'kalimat': 'pencitraan terus'}, {'label': 'neg', 'kalimat': 'Datang cuman pelanga plongo'}, {'label': 'neg', 'kalimat': 'lambatnya bertindak'}, {'label': 'neg', 'kalimat': 'Datang cuma foto doang.'}, {'label': 'pos', 'kalimat': 'Semangat pak kami semua mendukung bapak'}, {'label': 'pos', 'kalimat': 'Sehat terus pak. Tetap berjiwa besar dan selalu jadi sosok yang penyabar atas nyinyiran orang yg tak tau apa apa tentang dirimu'}, {'label': 'pos', 'kalimat': 'Masyaallah pak.. Fighting.'}]


<p>Berikut proses training Naive Bayes</p>

In [105]:
for i in range(len(training_data)):
    pos = 0
    neg = 0
    
    if training_raw_data[i]['label'] == 'pos':
        pos = 1
    else: 
        neg = 1
    
    for j in range(len(training_data[i])):
        add_into_corpus({'kata': training_data[i][j], 'pos': pos, 'neg': neg})
        

for d in nb_corpus:
    print(d['kata'],": ",d['pos']," ",d['neg'])

segala :  0   1
foto :  0   2
biar :  0   1
apa :  2   1
citra :  0   1
terus :  1   1
datang :  0   2
cuman :  0   1
pelanga :  0   1
plongo :  0   1
lambat :  0   1
tindak :  0   1
cuma :  0   1
doang :  0   1
semangat :  1   0
pak :  3   0
kami :  1   0
semua :  1   0
dukung :  1   0
bapak :  1   0
sehat :  1   0
tetap :  1   0
jiwa :  1   0
besar :  1   0
dan :  1   0
selalu :  1   0
jadi :  1   0
sosok :  1   0
yang :  1   0
sabar :  1   0
atas :  1   0
nyinyir :  1   0
orang :  1   0
yg :  1   0
tak :  1   0
tau :  1   0
tentang :  1   0
diri :  1   0
masyaallah :  1   0
fighting :  1   0


<p>Berikut hasil dari proses Naive Bayes terhadap seluruh data</p>

In [106]:
for word in stem_word:
    naivebayes(word)

['segala', 'foto', 'biar', 'apa']       negative
['citra', 'terus']       negative
['datang', 'cuman', 'pelanga', 'plongo']       negative
['lambat', 'tindak']       negative
['datang', 'cuma', 'foto', 'doang']       negative
['semangat', 'pak', 'kami', 'semua', 'dukung', 'bapak']       positive
['sehat', 'terus', 'pak', 'tetap', 'jiwa', 'besar', 'dan', 'selalu', 'jadi', 'sosok', 'yang', 'sabar', 'atas', 'nyinyir', 'orang', 'yg', 'tak', 'tau', 'apa', 'apa', 'tentang', 'diri']       positive
['kalo', 'emang', 'indonesia', 'mau', 'maju', 'pikir', 'warga', 'ga', 'harus', 'sempit', 'ya', 'masa', 'presiden', 'sendiri', 'ngebakar', 'wilayah', 'sendiri']       neutral
['wkwkwkwk', 'mantap', 'jok', 'kali', 'ini', 'akting', 'gak', 'sendiri', 'nih', 'yeee']       neutral
['kalau', 'foto', 'foto', 'doang', 'semua', 'orang', 'jugha', 'bisa', 'kayak', 'gitu', 'di', 'bilang', 'prestasi', 'haha', 'lucu', 'negriku']       negative
['mundur', 'saja', 'pakgak', 'kan', 'bisa', 'bapak', 'urus', 'negara', 

<p>Berikut keadaan corpus Naive Bayes setelah pemrosesan</p>

In [107]:
for d in nb_corpus:
    print(d['kata'],": ",d['pos']," ",d['neg'])

segala :  0   2
foto :  0   7
biar :  0   2
apa :  5   1
citra :  0   6
terus :  7   7
datang :  0   6
cuman :  0   3
pelanga :  0   2
plongo :  0   2
lambat :  0   2
tindak :  0   2
cuma :  0   4
doang :  0   4
semangat :  10   0
pak :  25   0
kami :  3   0
semua :  8   0
dukung :  2   0
bapak :  11   0
sehat :  8   0
tetap :  2   0
jiwa :  2   0
besar :  2   0
dan :  6   0
selalu :  2   0
jadi :  3   0
sosok :  2   0
yang :  3   0
sabar :  4   0
atas :  4   0
nyinyir :  2   0
orang :  3   0
yg :  5   0
tak :  2   0
tau :  3   0
tentang :  2   0
diri :  5   0
masyaallah :  2   0
fighting :  2   0
kalo :  1   1
emang :  1   1
indonesia :  7   7
mau :  3   3
maju :  2   2
pikir :  1   1
warga :  2   2
ga :  3   3
harus :  2   2
sempit :  1   1
ya :  3   3
masa :  1   1
presiden :  10   10
sendiri :  3   3
ngebakar :  1   1
wilayah :  1   1
wkwkwkwk :  1   1
mantap :  1   1
jok :  1   1
kali :  1   1
ini :  5   5
akting :  1   1
gak :  3   3
nih :  1   1
yeee :  1   1
kalau :  1   1
jugh

# K-Flod